# WEEK 1 CAPSTONE PROGRESSS
This is the first phase of the model development progress, in this phase we will try to design the base architecture of the model for our 'UrKins" application

In [ ]:
import tensorflow as tf 
import pandas as pd 
from tensorflow.keras.applications import MobileNetV2   
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D 
from tensorflow.keras.models import Model

In [ ]:
# Feature engineering
# Numeric feature columns
numeric_columns = ['age']
feature_columns = []
for header in numeric_columns:
    numeric_feature_column = tf.feature_column.numeric_column(header)
    feature_columns.append(numeric_feature_column)

# Bucketized age column
age = tf.feature_column.numeric_column('age')
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# Categorical column for 'thal'
thal = tf.feature_column.categorical_column_with_vocabulary_list(key='thal', vocabulary_list=['fixed', 'normal', 'reversible'])
thal_one_hot = tf.feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# Embedding column for 'thal'
thal_embedding = tf.feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# Crossed feature column
crossed_feature = tf.feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = tf.feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

# Create feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


In [ ]:
# 1. Load and preprocess data
def preprocess_data(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    return dataset

In [ ]:
# Example DataFrame
data = {'age': [23, 45, 67, 34, 25], 'thal': ['fixed', 'normal', 'reversible', 'normal', 'fixed'], 'target': [1, 0, 1, 0, 1]}
df = pd.DataFrame(data)

train_dataset = preprocess_data(df)

In [ ]:
# 2. Load Pretrained MobileNet
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

In [ ]:
# 3. Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
output = Dense(5, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# 4. Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 5. Train the model
# Dummy data for example purposes
train_images = tf.random.normal([10, 224, 224, 3])
train_labels = tf.random.uniform([10, 5], maxval=2, dtype=tf.int32)

model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

In [ ]:
# 6. Evaluate the model
test_images = tf.random.normal([2, 224, 224, 3])
test_labels = tf.random.uniform([2, 5], maxval=2, dtype=tf.int32)

model.evaluate(test_images, test_labels)

In [ ]:
# 7. Fine-tuning (Optional)
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

In [ ]:
# 8. Save the model
model.save('skin_condition_detector.h5')